In [1]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import sys

In [2]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform


dataset = rasterio.open("Data/tif_files/africa.tif")

In [5]:
import gdal
import osr
# open the dataset and get the geo transform matrix
ds = gdal.Open('Data/tif_files/africa.tif')
xoffset, px_w, rot1, yoffset, px_h, rot2 = ds.GetGeoTransform()
print(xoffset, px_w, rot1, yoffset, px_h, rot2)

-60.002083333 0.0041666667 0.0 75.002083333 0.0 -0.0041666667


In [6]:
band1 = dataset.read(1)
targets = (lambda x: x>0.1)(band1)
targets = targets*1

In [ ]:
coordinates = []
for i in range(dataset.height):
    for j in range(dataset.width):
        if targets[i][j]:
            posX = px_w * j + xoffset
            posY = rot2 * i + yoffset


            # shift to the center of the pixel
            posX += px_w / 2.0
            posY -= px_w / 2.0

            #get CRS from dataset 
            crs = osr.SpatialReference()
            crs.ImportFromWkt(ds.GetProjectionRef())
            # create lat/long crs with WGS84 datum
            crsGeo = osr.SpatialReference()
            crsGeo.ImportFromEPSG(4326) # 4326 is the EPSG id of lat/long crs 
            t = osr.CoordinateTransformation(crs, crsGeo)
            lat, long, z = t.TransformPoint(posY, posX)
            
            coordinates.append((lat, long))
    

df = pd.DataFrame(coordinates, columns =['lat', 'long'])
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.long, df.lat), crs="EPSG:4326")
gdf['id'] = df.groupby(['lat','long']).ngroup()

In [ ]:
gdf.to_file("Data/lightpoints.geojson", driver='GeoJSON')

In [3]:
import gridfinder as gf
uganda_guess = "Data/mv_lcfilter/UGA.tif"

In [3]:
guess,affine = gf.threshold(uganda_guess, 0.5)
print(type(guess))

<class 'numpy.ndarray'>


In [4]:
#guess_skel = gf.thin(guess)
guess_gdf=gf.raster_to_lines(uganda_guess)

In [4]:
guess_gdf.to_file("Uganda_grid.geojson", driver="GeoJSON")

NameError: name 'guess_gdf' is not defined

In [5]:
uga_grid = "Data/classification/ugandagrid.geojson"
from rasterio.features import rasterize


def true_positives(guesses, truths):
    """Calculate true positives, used by accuracy().

    Parameters
    ----------
    guesses : numpy array
        Output from model.
    truths : numpy array
        Truth feature converted to array.

    Returns
    -------
    tp : float
        Ratio of true positives.
    """

    yes_guesses = 0
    yes_guesses_correct = 0
    rows = guesses.shape[0]
    cols = guesses.shape[1]

    for x in range(0, rows):
        for y in range(0, cols):
            guess = guesses[x, y]
            truth = truths[x, y]
            if guess == 1:
                yes_guesses += 1
                if guess == truth:
                    yes_guesses_correct += 1

    tp = yes_guesses_correct / yes_guesses

    return tp


def false_negatives(guesses, truths):
    """Calculate false negatives, used by accuracy().

    Parameters
    ----------
    guesses : numpy array
        Output from model.
    truths : numpy array
        Truth feature converted to array.

    Returns
    -------
    fn : float
        Ratio of false negatives.
    """

    actual_grid = 0
    actual_grid_missed = 0

    rows = guesses.shape[0]
    cols = guesses.shape[1]

    for x in range(0, rows):
        for y in range(0, cols):
            guess = guesses[x, y]
            truth = truths[x, y]

            if truth == 1:
                actual_grid += 1
                if guess != truth:
                    found = False
                    for i in range(-5, 6):
                        for j in range(-5, 6):
                            if i == 0 and j == 0:
                                continue

                            shift_x = x + i
                            shift_y = y + j
                            if shift_x < 0 or shift_y < 0:
                                continue
                            if shift_x >= rows or shift_y >= cols:
                                continue

                            other_guess = guesses[shift_x, shift_y]
                            if other_guess == 1:
                                found = True
                    if not found:
                        actual_grid_missed += 1

    fn = actual_grid_missed / actual_grid

    return fn


def flip_arr_values(arr):
    """Simple helper function used by accuracy()"""

    arr[arr == 1] = 2
    arr[arr == 0] = 1
    arr[arr == 2] = 0
    return arr


def accuracy(grid_in, guess_in, aoi_in, country, buffer_amount=0.01):
    """Measure accuracy against a specified grid 'truth' file.

    Parameters
    ----------
    grid_in : str, Path
        Path to vector truth file.
    guess_in : str, Path
        Path to guess output from guess2geom.
    aoi_in : str, Path
        Path to AOI feature.
    buffer_amount : float, optional (default 0.01.)
        Leeway in decimal degrees in calculating equivalence.
        0.01 DD equals approximately 1 mile at the equator.
    """
    if isinstance(aoi_in, gpd.GeoDataFrame):
        aoi = aoi_in
    else:
        aoi = gpd.read_file(aoi_in)
        aoi = aoi[aoi["ADM0_A3"] == country]

    grid_masked = gpd.read_file(grid_in, mask=aoi)
    grid = gpd.sjoin(grid_masked, aoi, how="inner", op="intersects")
    grid = grid[grid_masked.columns]

    grid_buff = grid.buffer(buffer_amount)

    guesses_reader = rasterio.open(guess_in)
    guesses = guesses_reader.read(1)

    grid_for_raster = [(row.geometry) for _, row in grid.iterrows()]
    grid_raster = rasterize(
        grid_for_raster,
        out_shape=guesses_reader.shape,
        fill=1,
        default_value=0,
        all_touched=True,
        transform=guesses_reader.transform,
    )
    grid_buff_raster = rasterize(
        grid_buff,
        out_shape=guesses_reader.shape,
        fill=1,
        default_value=0,
        all_touched=True,
        transform=guesses_reader.transform,
    )

    grid_raster = flip_arr_values(grid_raster)
    grid_buff_raster = flip_arr_values(grid_buff_raster)

    tp = true_positives(guesses, grid_buff_raster)
    fn = false_negatives(guesses, grid_raster)

    return tp, fn


tp, fn = accuracy(uga_grid, uganda_guess, "admin_boundaries.gpkg", "UGA")

/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
<ipython-input-5-52ee7db2c2dd>:128: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_buff = grid.buffer(buffer_amount)


In [6]:
print(tp,fn)

0.7049542993537871 0.4485729746229517


In [2]:
import sys
from pathlib import Path
from argparse import ArgumentParser
import numpy as np
from rasterio.warp import reproject, Resampling

import geopandas as gpd
import rasterio

from gridfinder import clip_raster, save_raster


def make_same_as(curr_arr, curr_aff, curr_crs, dest_arr_like, dest_affine, dest_crs):
    """

    """

    dest_arr = np.empty_like(dest_arr_like)

    with rasterio.Env():
        reproject(
            source=curr_arr,
            destination=dest_arr,
            src_transform=curr_aff,
            dst_transform=dest_affine,
            src_crs=curr_crs,
            dst_crs=dest_crs,
            resampling=Resampling.max-Resampling.min,
        )

    return dest_arr



def clip_all(raster_in, admin_in, raster_shape_dir, dir_out, code="ADM0_A3"):
    raster_in = Path(raster_in).expanduser()
    admin_in = Path(admin_in).expanduser()
    raster_shape_dir = Path(raster_shape_dir).expanduser()
    dir_out = Path(dir_out).expanduser()

    admin = gpd.read_file(admin_in)
    countries = admin[code].tolist()

    for c in countries:
        print(f"Doing {c}")
        c_out = dir_out / f"{c}.tif"

        aoi = admin[admin[code] == c]
        try:
            arr, aff, crs = gf.clip_raster(raster_in, aoi)
        except ValueError:
            print("No input data for this AOI - skipped")
            continue
        targets_in = raster_shape_dir / f"{c}.tif"
        targets_rd = rasterio.open(targets_in)
        dest_arr_like = targets_rd.read(1)
        dest_affine = targets_rd.transform
        dest_crs = targets_rd.crs

        new_arr = make_same_as(arr, aff, crs, dest_arr_like, dest_affine, dest_crs)

        gf.save_raster(c_out, new_arr, dest_affine, dest_crs)
#clip_all("Data/lccs.tif", "admin_boundaries.gpkg", "Data/targets_filt/", "Data/LandCover_rs/")

Doing AGO


/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


NameError: name 'gf' is not defined

In [17]:
uganda_cover = rasterio.open("Data/LandCover_rs/UGA.tif")
uganda_covers = uganda_cover.read(1)
print(np.unique(uganda_covers))


uganda_cover1 = rasterio.open("Data/LandCover/UGA.tif")
uganda_covers1 = uganda_cover1.read(1)
print(np.unique(uganda_covers1))


[  0.  10.  11.  12.  20.  30.  40.  50.  60.  61.  62.  70.  90. 100.
 110. 120. 122. 130. 160. 170. 180. 190. 200. 201. 210.]
[  0.  10.  11.  12.  20.  30.  40.  50.  60.  61.  62.  70.  90. 100.
 110. 120. 122. 130. 150. 160. 170. 180. 190. 200. 201. 210.]


In [18]:
uganda_covers.shape

(1392, 1335)

In [19]:
collections = {}
for i in range(uganda_covers.shape[0]):
    for j in range(uganda_covers.shape[1]):
        if uganda_covers[i,j] in collections.keys():
            collections[uganda_covers[i,j]] +=1
        else:
            collections[uganda_covers[i,j]] =1
print(collections)

{0.0: 663825, 10.0: 398440, 30.0: 92765, 60.0: 14647, 130.0: 27529, 40.0: 33408, 62.0: 194523, 50.0: 60070, 120.0: 93689, 100.0: 8140, 122.0: 1869, 11.0: 53603, 180.0: 16258, 210.0: 187381, 110.0: 955, 190.0: 2222, 20.0: 7707, 170.0: 328, 61.0: 318, 160.0: 570, 12.0: 52, 200.0: 12, 201.0: 2, 90.0: 3, 70.0: 4}


In [20]:
uganda_truth = rasterio.open("Data/classification/Uganda_grid.tif")
uganda_truths = uganda_truth.read(1)

In [21]:
collections2 = {}
for i in range(uganda_covers.shape[0]):
    for j in range(uganda_covers.shape[1]):
        if uganda_truths[i,j]:
            if uganda_covers[i,j] in collections2.keys():
                collections2[uganda_covers[i,j]] +=1
            else:
                collections2[uganda_covers[i,j]] =1
print(collections2)

{10.0: 39556, 40.0: 1209, 122.0: 45, 120.0: 1617, 130.0: 787, 60.0: 373, 62.0: 3411, 30.0: 4964, 50.0: 768, 190.0: 1940, 210.0: 137, 180.0: 318, 20.0: 582, 11.0: 6013, 61.0: 1, 100.0: 517, 170.0: 5, 160.0: 6, 0.0: 647, 12.0: 4, 110.0: 8}


In [35]:
collections2 = {}
for i in range(uganda_covers.shape[0]):
    for j in range(uganda_covers.shape[1]):
        if uganda_truths[i,j]:
            if uganda_covers[i,j] in collections2.keys():
                collections2[uganda_covers[i,j]] +=1
            else:
                collections2[uganda_covers[i,j]] =1
for key in collections2.keys():
    collections2[key]/=collections[key]
for key in collections2.keys():
    
    collections2[key] = 1/collections2[key]
print(collections2)


{10.0: 10.072808170694712, 40.0: 27.632754342431763, 122.0: 41.53333333333333, 120.0: 57.9400123685838, 130.0: 34.97966963151207, 60.0: 39.26809651474531, 62.0: 57.02814423922604, 30.0: 18.6875503626108, 50.0: 78.21614583333333, 190.0: 1.1453608247422682, 210.0: 1367.7445255474452, 180.0: 51.12578616352201, 20.0: 13.242268041237113, 11.0: 8.914518543156493, 61.0: 318.0, 100.0: 15.744680851063828, 170.0: 65.60000000000001, 160.0: 95.0, 0.0: 1026.0046367851623, 12.0: 13.0, 110.0: 119.37499999999999}


In [24]:
ken_cover = rasterio.open("Data/LandCover_rs/KEN.tif")
ken_covers = ken_cover.read(1)

kenya_truth = rasterio.open("Data/classification/kenya_grid/kenya_grid.tif")
kenya_truths = kenya_truth.read(1)

In [34]:
col = {}
for i in range(ken_covers.shape[0]):
    for j in range(ken_covers.shape[1]):
        if ken_covers[i,j] in col.keys():
            col[ken_covers[i,j]] +=1
        else:
            col[ken_covers[i,j]] =1
print(sorted(col.keys()))

[0.0, 10.0, 11.0, 12.0, 20.0, 30.0, 40.0, 50.0, 60.0, 61.0, 62.0, 90.0, 100.0, 110.0, 120.0, 122.0, 130.0, 150.0, 152.0, 153.0, 160.0, 170.0, 180.0, 190.0, 200.0, 201.0, 202.0, 210.0]


In [50]:
"""
Need to initialize for 70,71,72, 80,81,82, 121, 140, 151, 
"""
liste = [70,71,72, 80, 81, 82, 121, 140,151]

In [51]:
col2 = {}
for i in range(ken_covers.shape[0]):
    for j in range(ken_covers.shape[1]):
        if kenya_truths[i,j]:
            if ken_covers[i,j] in col2.keys():
                col2[ken_covers[i,j]] +=1
            else:
                col2[ken_covers[i,j]] =1
for key in col2.keys():
    col2[key]/=col[key]
for key in col2.keys():
    col2[key]=1/col2[key]
print(col2)

{40.0: 12.37403834412016, 130.0: 99.25122139367446, 62.0: 60.428011753183156, 100.0: 46.45558157689305, 30.0: 8.240224526686328, 120.0: 65.35256087321578, 10.0: 6.27778461322131, 20.0: 7.258241758241758, 190.0: 1.1172140430351076, 11.0: 4.8442326024785505, 50.0: 23.865747126436784, 110.0: 294.09722222222223, 150.0: 260.3962264150943, 200.0: 1090.7872340425533, 153.0: 16.400000000000002, 180.0: 203.08379888268155, 152.0: 74.07142857142857, 60.0: 10.336581970066133, 170.0: 106.0, 122.0: 4.620689655172414, 210.0: 391.139175257732, 201.0: 1964.7272727272727, 90.0: 33.49625935162095, 12.0: 5.044642857142858, 61.0: 17.956521739130434, 160.0: 15.865546218487395, 0.0: 346586.6}


In [52]:
for key, values in col2.items():
    try:
        col2[key] = values + collections2[key]
    except:
        collections2[key] = sum(collections2.values())/len(collections2.keys())
        col2[key] = values + collections2[key]
    
"""
Now to have this as around the same bias as roads, we want to have the total number a lot smaller
"""
col2[0]=0
for i in liste:
    col[i] = max(col2.values())
fsums = sum(col2.values())
for key, values in col2.items():
    col2[key] = 10*col2[key]/fsums
col2[0]=10000
print(col2)

{40.0: 0.04851435495794723, 130.0: 0.16277548526669194, 62.0: 0.14243355343322026, 100.0: 0.07542733139219432, 30.0: 0.032654045512780956, 120.0: 0.1495111019969868, 10.0: 0.019827594486433186, 20.0: 0.024860003575470635, 190.0: 0.0027437180759351362, 11.0: 0.016684590091666538, 50.0: 0.12378990809463708, 110.0: 0.5013983078149369, 150.0: 0.5155840016273311, 200.0: 1.5225601022500612, 153.0: 0.219701259177932, 180.0: 0.30826800186833286, 152.0: 0.289636686858109, 60.0: 0.0601532594337645, 170.0: 0.2080912453045048, 122.0: 0.05596881188514511, 210.0: 2.132915499104576, 201.0: 2.582346063621683, 90.0: 0.24043308842343075, 12.0: 0.021881889296141575, 61.0: 0.40739866536634955, 160.0: 0.13444143108373643, 0.0: 10000}


In [38]:
def replace_with_dict2(ar, dic):
    # Extract out keys and values
    k = np.array(list(dic.keys()))
    v = np.array(list(dic.values()))

    # Get argsort indices
    sidx = k.argsort()

    ks = k[sidx]
    vs = v[sidx]
    return vs[np.searchsorted(ks,ar)]

print(replace_with_dict2(uganda_covers, col2))

[[10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 ...
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]
 [10000. 10000. 10000. ... 10000. 10000. 10000.]]


In [59]:
import gridfinder as gf

def make_new_costs(path_costs, path_lc, dir_out, admin_in,code="ADM0_A3"):
    
    path_costs = Path(path_costs).expanduser()
    admin_in = Path(admin_in).expanduser()
    path_lc = Path(path_lc).expanduser()
    dir_out = Path(dir_out).expanduser()
    
    admin = gpd.read_file(admin_in)
    countries = admin[code].tolist()

    for c in countries:
        print(f"Doing {c}")
        c_out = dir_out / f"{c}.tif"
        try:

            cost_in = path_costs / f"{c}.tif"
            cost_rd = rasterio.open(cost_in)
            dest_arr_like = cost_rd.read(1)
            dest_affine = cost_rd.transform
            dest_crs = cost_rd.crs

            lcc_in = path_lc / f"{c}.tif"
            lcc_rd = rasterio.open(lcc_in)
            lcc_arr = cost_rd.read(1)
            
            lcc_arr = replace_with_dict2(lcc_arr, col2)

            #new_arr = np.multiply(dest_arr_like, lcc_arr)

            gf.save_raster(c_out, lcc_arr, dest_affine, dest_crs)
        except:
            pass
        
        
make_new_costs("Data/costs", "Data/LandCover_rs", "Data/costs_lc2", "admin_boundaries.gpkg")

/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Doing AGO
Doing MAR
Doing BWA
Doing SAU
Doing CPV
Doing EGY
Doing LBR
Doing MWI
Doing LBY
Doing CAF
Doing TUN
Doing MOZ
Doing GIB
Doing CIV
Doing NAM
Doing COG
Doing RWA
Doing COD
Doing COM
Doing JOR
Doing ZAF
Doing BDI
Doing GIN
Doing GAB
Doing BFA
Doing SAH
Doing MLI
Doing BEN
Doing UGA
Doing MDG
Doing TCD
Doing PSX
Doing ESP
Doing GNQ
Doing NGA
Doing LSO
Doing GHA
Doing YEM
Doing ISR
Doing ZWE
Doing ETH
Doing DJI
Doing DZA
Doing GMB
Doing MRT
Doing CMR
Doing SWZ
Doing NER
Doing SOL
Doing SOM
Doing SDN
Doing GNB
Doing SDS
Doing SLE
Doing ERI
Doing KEN
Doing TZA
Doing TGO
Doing STP
Doing SEN
Doing ZMB
